In [1]:
import hdfs3

In [2]:
hdfs = hdfs3.HDFileSystem(host='54.86.159.15')

In [3]:
from distributed import Executor, progress, wait

In [4]:
from distributed.hdfs import read_bytes, write_bytes

In [ ]:
e = Executor('54.86.159.15:8786')

In [6]:
e.restart()

Generate a list of dicts with the parameters to test using Scikit-learn ParameterGrid

In [7]:
from sklearn.grid_search import ParameterGrid

In [8]:
import numpy as np

In [9]:
param_grid = {'learning_rate': np.linspace(0.1, 0.8, num=8), 'steps': [1000]}
params_ = list(ParameterGrid(param_grid))

Use `scatter` to create a list of futures each future having one of the parameters set

In [10]:
params = e.scatter(params_)

Load the train and test data in all the workers

In [ ]:
def load_train():
    import numpy as np
    from io import StringIO
    
    with hdfs.open('/tmp/mnist_train.csv') as f:
        train = f.read()

    train = StringIO(train.decode('utf-8'))
    mnist_train = np.loadtxt(train, delimiter=',')
    return mnist_train

In [ ]:
def load_test():
    import numpy as np
    from io import StringIO
        
    with hdfs.open('/tmp/mnist_test.csv') as f:
        test = f.read()
    
    test = StringIO(test.decode('utf-8'))
    mnist_test = np.loadtxt(test, delimiter=',')
    return mnist_test

Create a big function that is going to get mapped to all the workers

In [11]:
def train(params, train_data, test_data):
    import skflow
    
    mnistX_train = train_data[:, 1:]
    mnistY_train = train_data[:, 0]
    mnistX_test = test_data[:, 1:]
    mnistY_test = test_data[:, 0]
    
    classifier = skflow.TensorFlowLinearClassifier(
            n_classes=10, batch_size=100, **params)
    classifier.fit(mnistX_train, mnistY_train)
    
    from sklearn.metrics import accuracy_score
    score = accuracy_score(classifier.predict(mnistX_test), mnistY_test)
    return score

In [13]:
results = e.map(train, params, traindata=future_traindata, testdata=future_testdata)

Wait for the tasks to be done

In [14]:
progress(results)

In [ ]:
results

Get the accuracy scores

In [ ]:
[a for a in results]